In [1]:
#load libraries
require(tidyr)
require(dplyr)

Loading required package: tidyr

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
#load Colony_Data.csv
colony=read.csv("~/Documents/SCTLD/SCTLD_samples/Sample_Data/CBC_ColonyData.csv")

In [3]:
#isolate immune samples from colony data
colony_immune=colony[colony$immune_y.n=="y",]
colony_immune=as.vector(paste0(colony_immune$TransectNum,"_", colony_immune$NewTagNum))

In [4]:
#load metagenomics data
metagenomics=read.csv("~/Documents/SCTLD/SCTLD_samples/Sample_Data/Metagenomics_Tracker_Belize.csv")

In [5]:
#load sample_data
sample=read.csv("~/Documents/SCTLD/SCTLD_samples/Sample_Data/CBC_samples.csv")

# sample_dna add colony column to sample data
sample$colony= colony= c(paste0(sample$TransectNum, "_", sample$NewTagNum))  

# only utilize EtOH and RNALater samples
sample_DNA=sample[sample$Sample_type=="Core_EtOH"|sample$Sample_type=="Core_RNAlater",]

#view sample_dna
head(sample_DNA)

,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,⋯,Sample_type,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes,colony
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,92023,BEL,CBC,9/27/23,CBC30N,1,,1,SSID,,⋯,Core_RNAlater,185,Diseased_Margin,only margin sample available,092023_BEL_CBC_T1_185_SSID,UML_NARWHAL_R1_B10,,,,1_1
3,92023,BEL,CBC,9/25/23,CBC30N,1,,2,PAST,,⋯,Core_RNAlater,171,Healthy,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,1_2
4,92023,BEL,CBC,9/25/23,CBC30N,1,,3,SSID,,⋯,Core_RNAlater,173,Healthy,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,1_3
5,92023,BEL,CBC,9/25/23,CBC30N,1,,12,PSTR,,⋯,Core_RNAlater,177,Healthy,No CL,092023_BEL_CBC_T1_177_PSTR,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,1_12
6,92023,BEL,CBC,9/25/23,CBC30N,1,,13,PAST,,⋯,Core_RNAlater,175,Healthy,No CL,092023_BEL_CBC_T1_175_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,R2_B15 EXTRACTED TWICE,1_13
7,92023,BEL,CBC,9/27/23,CBC30N,1,,5,SSID,,⋯,Core_RNAlater,188,Diseased_Tissue,,092023_BEL_CBC_T1_188_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B11,,,1_5


In [6]:
#changing X column to Tubelabel_species
names(metagenomics)[names(metagenomics) == 'X'] <- 'Tubelabel_species'

In [7]:
#the columns that I want to isolate from metagenomics tracker
metagenomics_PCR = metagenomics[c("Tubelabel_species", "Health_Status", "Date_Extracted", "Raw_ng_ul", "Extraction_physical_location", "Date_Enriched")]

In [8]:
head(metagenomics_PCR)

,Tubelabel_species,Health_Status,Date_Extracted,Raw_ng_ul,Extraction_physical_location,Date_Enriched
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,052022_BEL_CBC_T2_45_PSTR,Diseased_Margin,10_10_2023,22.8,UML_NARWHAL_R2_B3,10_17_2023
2,052022_BEL_CBC_T2_46_PSTR,Diseased_Tissue,10_10_2023,63.9,UML_NARWHAL_R2_B3,10_17_2023
3,052022_BEL_CBC_T2_59_OFAV,Healthy,10_3_2023,73.7,UML_NARWHAL_R2_B3,10_16_2023
4,042024_BEL_CBC_T1_925_PAST,Healthy,8_19_2024,5.46,UML_NARWHAL_R2_B29,9_28_2024
5,052022_BEL_CBC_T2_72_OFAV,Healthy,10_3_2023,47.6,UML_NARWHAL_R2_B3,10_11_23
6,052022_BEL_CBC_T2_71_OFAV,Healthy,10_3_2023,98.8,UML_NARWHAL_R2_B3,10_16_2023


In [9]:
#info I am using to create ties between sample_dna and metagenomics_PCR
sample_DNA=sample_DNA[c("Tubelabel_species", "colony")]

In [10]:
#only including immune samples in sample_DNA
immune_sample_DNA=sample_DNA[sample_DNA$colony %in% colony_immune,]

In [11]:
head(immune_sample_DNA)

,Tubelabel_species,colony
,<chr>,<chr>
3,092023_BEL_CBC_T1_171_PAST,1_2
4,092023_BEL_CBC_T1_173_SSID,1_3
5,092023_BEL_CBC_T1_177_PSTR,1_12
6,092023_BEL_CBC_T1_175_PAST,1_13
12,092023_BEL_CBC_T1_172_PAST,1_21
13,092023_BEL_CBC_T1_174_MCAV,1_24


In [12]:
nrow(metagenomics_PCR)

[1] 524

In [13]:
nrow(immune_sample_DNA)

[1] 368

In [14]:
unique(immune_sample_DNA$Tubelabel_species)

[1] "092023_BEL_CBC_T1_171_PAST"      "092023_BEL_CBC_T1_173_SSID"     
  [3] "092023_BEL_CBC_T1_177_PSTR"      "092023_BEL_CBC_T1_175_PAST"     
  [5] "092023_BEL_CBC_T1_172_PAST"      "092023_BEL_CBC_T1_174_MCAV"     
  [7] "092023_BEL_CBC_T1_178_OANN"      "092023_BEL_CBC_T2_183_MCAV"     
  [9] "092023_BEL_CBC_T2_182_PAST"      "092023_BEL_CBC_T2_184_MCAV"     
 [11] "092023_BEL_CBC_T2_181_PAST"      "092023_BEL_CBC_T2_175_PAST"     
 [13] "092023_BEL_CBC_T2_176_MCAV"      "092023_BEL_CBC_T2_174_SSID"     
 [15] "092023_BEL_CBC_T2_173_SSID"      "092023_BEL_CBC_T2_172_OFAV"     
 [17] "092023_BEL_CBC_T2_171_OFAV"      "092023_BEL_CBC_T2_179_SSID"     
 [19] "092023_BEL_CBC_T3_176_MCAV"      "092023_BEL_CBC_T3_177_SSID"     
 [21] "092023_BEL_CBC_T3_178_PAST"      "092023_BEL_CBC_T3_181_PAST"     
 [23] "092023_BEL_CBC_T3_175_MCAV"      "092023_BEL_CBC_T3_172_PSTR"     
 [25] "092023_BEL_CBC_T3_182_MCAV"      "092023_BEL_CBC_T3_183_SSID"     
 [27] "092023_BEL_CBC_T3_174_OFAV"      "092023_BEL_CBC_T3_171_OFAV"     
 [29] "092023_BEL_CBC_T3_173_OFAV"      "092023_BEL_CBC_T3_195_PSTR"     
 [31] "092023_BEL_CBC_T4_91_MCAV"       "092023_BEL_CBC_T4_96_MCAV"      
 [33] "092023_BEL_CBC_T4_94_MCAV"       "092023_BEL_CBC_T4_95_OFAV"      
 [35] "092023_BEL_CBC_T4_93_PSTR"       "092023_BEL_CBC_T4_92_PSTR"      
 [37] "092023_BEL_CBC_T3_184_PAST"      "112023_BEL_CBC_T1_273_PAST"     
 [39] "112023_BEL_CBC_T1_275_SSID"      "112023_BEL_CBC_T1_278_PSTR"     
 [41] "112023_BEL_CBC_T1_274_PAST"      "112023_BEL_CBC_T1_276_MCAV"     
 [43] "112023_BEL_CBC_T1_277_OFAV/OANN" "112023_BEL_CBC_T2_310_MCAV"     
 [45] "112023_BEL_CBC_T2_309_PAST"      "112023_BEL_CBC_T2_311_MCAV"     
 [47] "112023_BEL_CBC_T2_308_PAST"      "112023_BEL_CBC_T2_305_PAST"     
 [49] "112023_BEL_CBC_T2_306_MCAV"      "112023_BEL_CBC_T2_313_SSID"     
 [51] "112023_BEL_CBC_T2_314_SSID"      "122022_BEL_CBC_T1_122_PSTR"     
 [53] "112023_BEL_CBC_T2_304_OFAV"      "112023_BEL_CBC_T3_359_PAST"     
 [55] "112023_BEL_CBC_T2_312_OFAV"      "122022_BEL_CBC_T1_150_PAST"     
 [57] "122022_BEL_CBC_T1_151_MCAV"      "122022_BEL_CBC_T1_152_PAST"     
 [59] "122022_BEL_CBC_T1_156_SSID"      "112023_BEL_CBC_T2_307_SSID"     
 [61] "122022_BEL_CBC_T1_158_PAST"      "122022_BEL_CBC_T2_100_PAST"     
 [63] "122022_BEL_CBC_T2_83_PAST"       "122022_BEL_CBC_T2_86_MCAV"      
 [65] "122022_BEL_CBC_T2_87_OFAV"       "122022_BEL_CBC_T2_88_MCAV"      
 [67] "122022_BEL_CBC_T2_90_SSID"       "122022_BEL_CBC_T2_92_MCAV"      
 [69] "122022_BEL_CBC_T2_93_OFAV"       "122022_BEL_CBC_T2_94_PAST"      
 [71] "122022_BEL_CBC_T2_96_SSID"       "122022_BEL_CBC_T3_116_PAST"     
 [73] "122022_BEL_CBC_T3_119_MCAV"      "122022_BEL_CBC_T3_120_SSID"     
 [75] "122022_BEL_CBC_T3_121_OFAV"      "122022_BEL_CBC_T3_123_PAST"     
 [77] "122022_BEL_CBC_T3_124_PAST"      "122022_BEL_CBC_T3_126_SSID"     
 [79] "122022_BEL_CBC_T3_127_OFAV"      "122022_BEL_CBC_T3_128_MCAV"     
 [81] "122022_BEL_CBC_T3_141_PSTR"      "122022_BEL_CBC_T3_142_MCAV"     
 [83] "122022_BEL_CBC_T3_151_PSTR"      "122022_BEL_CBC_T4_11_MCAV"      
 [85] "122022_BEL_CBC_T4_3_MCAV"        "122022_BEL_CBC_T4_34_PSTR"      
 [87] "122022_BEL_CBC_T4_38_MCAV"       "122022_BEL_CBC_T4_1_OFAV"       
 [89] "122022_BEL_CBC_T4_4_PSTR"        "052022_BEL_CBC_T1_62_MCAV"      
 [91] "052022_BEL_CBC_T2_59_OFAV"       "052022_BEL_CBC_T2_71_OFAV"      
 [93] "052022_BEL_CBC_T2_1_SSID"        "052022_BEL_CBC_T2_10_MCAV"      
 [95] "052022_BEL_CBC_T2_11_PAST"       "052022_BEL_CBC_T2_12_MCAV"      
 [97] "052022_BEL_CBC_T2_3_SSID"        "052022_BEL_CBC_T2_4_MCAV"       
 [99] "052022_BEL_CBC_T2_5_PAST"        "052022_BEL_CBC_T2_9_PAST"       
[101] "052022_BEL_CBC_T1_1_PAST"        "052022_BEL_CBC_T1_2_SSID"       
[103] "112023_BEL_CBC_T3_347_MCAV"      "052022_BEL_CBC_T1_34_PAST"      
[105] "122022_BEL_CBC_T1_123_OANN"      "052022_BEL_CBC_T1_53_PAST"      
[107] "052022_BEL_CBC_T1_54_MCAV"       "052022_BEL_CBC_T1_55_PSTR"      
[109] 

In [15]:
# Identify the incorrect labels that contain "Belize"
incorrect_names <- immune_sample_DNA$Tubelabel_species[grepl("Belize", immune_sample_DNA$Tubelabel_species)]
print(incorrect_names)

 [1] "022024_Belize_CBC_T1_771_PAST" "022024_Belize_CBC_T1_773_SSID"
 [3] "022024_Belize_CBC_T1_776_PSTR" "022024_Belize_CBC_T1_774_MCAV"
 [5] "022024_Belize_CBC_T2_810_SSID" "022024_Belize_CBC_T2_813_MCAV"
 [7] "022024_Belize_CBC_T2_812_PAST" "022024_Belize_CBC_T2_814_MCAV"
 [9] "022024_Belize_CBC_T2_811_PAST" "022024_Belize_CBC_T2_808_PAST"
[11] "022024_Belize_CBC_T2_809_MCAV" "022024_Belize_CBC_T2_805_SSID"
[13] "022024_Belize_CBC_T2_806_SSID" "022024_Belize_CBC_T2_807_SSID"
[15] "022024_Belize_CBC_T2_804_OFAV" "022024_Belize_CBC_T2_815_OFAV"
[17] "022024_Belize_CBC_T3_842_MCAV" "022024_Belize_CBC_T3_843_MCAV"
[19] "022024_Belize_CBC_T3_845_SSID" "022024_Belize_CBC_T3_847_PAST"
[21] "022024_Belize_CBC_T3_848_MCAV" "022024_Belize_CBC_T3_849_PSTR"
[23] "022024_Belize_CBC_T3_850_MCAV" "022024_Belize_CBC_T3_853_SSID"
[25] "022024_Belize_CBC_T3_854_PAST" "022024_Belize_CBC_T3_844_OFAV"
[27] "022024_Belize_CBC_T3_852_OFAV" "022024_Belize_CBC_T3_851_PSTR"
[29] "022024_Belize_CBC_T4_870_MCA

In [16]:
# Replace "Belize" with "BEL" for the problematic names
immune_sample_DNA$Tubelabel_species <- gsub("Belize", "BEL", immune_sample_DNA$Tubelabel_species)

In [17]:
# Check to ensure only "Belize" was replaced with "BEL"
print(immune_sample_DNA$Tubelabel_species[grepl("BEL", immune_sample_DNA$Tubelabel_species)])

  [1] "092023_BEL_CBC_T1_171_PAST"      "092023_BEL_CBC_T1_173_SSID"     
  [3] "092023_BEL_CBC_T1_177_PSTR"      "092023_BEL_CBC_T1_175_PAST"     
  [5] "092023_BEL_CBC_T1_172_PAST"      "092023_BEL_CBC_T1_174_MCAV"     
  [7] "092023_BEL_CBC_T1_178_OANN"      "092023_BEL_CBC_T2_183_MCAV"     
  [9] "092023_BEL_CBC_T2_182_PAST"      "092023_BEL_CBC_T2_184_MCAV"     
 [11] "092023_BEL_CBC_T2_181_PAST"      "092023_BEL_CBC_T2_175_PAST"     
 [13] "092023_BEL_CBC_T2_176_MCAV"      "092023_BEL_CBC_T2_174_SSID"     
 [15] "092023_BEL_CBC_T2_173_SSID"      "092023_BEL_CBC_T2_172_OFAV"     
 [17] "092023_BEL_CBC_T2_171_OFAV"      "092023_BEL_CBC_T2_179_SSID"     
 [19] "092023_BEL_CBC_T3_176_MCAV"      "092023_BEL_CBC_T3_177_SSID"     
 [21] "092023_BEL_CBC_T3_178_PAST"      "092023_BEL_CBC_T3_181_PAST"     
 [23] "092023_BEL_CBC_T3_175_MCAV"      "092023_BEL_CBC_T3_172_PSTR"     
 [25] "092023_BEL_CBC_T3_182_MCAV"      "092023_BEL_CBC_T3_183_SSID"     
 [27] "092023_BEL_CBC_T3_174_OFAV"    

In [18]:
# inner join 
immune_metagenomics_PCR <- merge(x=metagenomics_PCR,y=immune_sample_DNA, 
                by="Tubelabel_species")
head(immune_metagenomics_PCR)

,Tubelabel_species,Health_Status,Date_Extracted,Raw_ng_ul,Extraction_physical_location,Date_Enriched,colony
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,022024_BEL_CBC_T1_771_PAST,Healthy,11_22_2024,23.8,UML_NARWHAL_R2_B13,,1_2
2,022024_BEL_CBC_T1_773_SSID,Healthy,11_22_2024,61.8,UML_NARWHAL_R2_B13,,1_3
3,022024_BEL_CBC_T1_774_MCAV,Healthy,11_22_2024,31.2,UML_NARWHAL_R2_B13,,1_24
4,022024_BEL_CBC_T1_775_OANN,Healthy,11_22_2024,13.8,UML_NARWHAL_R2_B13,,1_25
5,022024_BEL_CBC_T1_776_PSTR,Healthy,11_22_2024,20.2,UML_NARWHAL_R2_B13,,1_12
6,022024_BEL_CBC_T2_804_OFAV,Healthy,11_22_2024,86.8,UML_NARWHAL_R2_B13,,2_76


In [19]:
length(unique(immune_metagenomics_PCR$colony))

[1] 37

In [20]:
nrow(immune_sample_DNA)

[1] 368

In [21]:
nrow(immune_metagenomics_PCR)

[1] 340

In [22]:
immune_unmatched=anti_join(immune_sample_DNA, immune_metagenomics_PCR, by='Tubelabel_species')
nrow(immune_unmatched)
(immune_unmatched)

[1] 33

Tubelabel_species,colony
<chr>,<chr>
052022_BEL_CBC_T3_25_PAST,3_34
012024_BEL_CBC_T4_661_PSTR,4_98
012024_BEL_CBC_T4_663_PSTR,4_79
012024_BEL_CBC_T4_665_MCAV,4_76
012024_BEL_CBC_T4_667_OFAV,4_78
012024_BEL_CBC_T4_669_MCAV,4_30
012024_BEL_CBC_T1_557_SSID,1_3
012024_BEL_CBC_T1_559_MCAV,1_24
012024_BEL_CBC_T1_563_PSTR,1_12


In [23]:
#manually corrected for these unmatched values in metagenomics tracker and CBC_samples
#mostly my own typos
#find 052022_BEL_CBC_T3_25_PAST extraction
#do we want to extract 012024?

In [25]:
#make spreadsheet with all immune metagenomics 
write.csv(immune_metagenomics_PCR, file="~/Documents/SCTLD/Coral_Code/NSF_Rapid/CBC_PCR_Plan/immune_metagenomics_PCR.csv")

In [ ]:
# Step 1: Load your data
immune_metagenomics_PCR <- read.csv("immune_metagenomics_PCR.csv")

# Step 2: Shuffle the rows
randomized_immune_metagenomics_PCR <- immune_metagenomics_PCR[sample(nrow(immune_metagenomics_PCR)), ]

# Step 3: Save the shuffled data to a new .csv file
write.csv(randomized_immune_metagenomics_PCR, "~/Documents/SCTLD/Coral_Code/NSF_Rapid/CBC_PCR_Plan/randomized_immune_metagenomics_PCR.csv", row.names = FALSE)